#Environment Setup

In [ ]:
!which python
!python --version
%env PYTHONPATH=
!pip install virtualenv
!virtualenv myenv
!wget https://repo.anaconda.com/miniconda/Miniconda3-latest-Linux-x86_64.sh
!chmod +x Miniconda3-latest-Linux-x86_64.sh
!./Miniconda3-latest-Linux-x86_64.sh -b -f -p /usr/local
!conda install -q -y --prefix /usr/local python=3.8 ujson
import sys
sys.path.append('/usr/local/lib/python3.8/site-packages/')
import os
os.environ['CONDA_PREFIX'] = '/usr/local/envs/myenv'
!python --version


#Clone repository

In [ ]:
!git clone https://github.com/sergiuabed/SICK_Summarization.git
ROOT = "/content/SICK_Summarization/extension_2"
%cd $ROOT

#Install requirements

In [ ]:
!conda create -n sick python=3.8 -y
!conda activate sick
!pip install torch==1.10.1+cu113 torchvision==0.11.2+cu113 torchaudio==0.10.1 -f https://download.pytorch.org/whl/cu113/torch_stable.html
!pip install -r SICK_on_tweetsumm/requirements.txt
!pip install -U spacy
!python -m spacy download en_core_web_sm


#Test SICK on Tweetsumm

##Get data and commonsense

In [ ]:

!unzip -o Tweetsumm/tweetsumm_comet.zip -d SICK_on_tweetsumm/data
!unzip -o Tweetsumm/tweetsumm_clean.zip -d SICK_on_tweetsumm/data
!mv SICK_on_tweetsumm/data/dialogue_data/ SICK_on_tweetsumm/data/tweetsumm_data/

##Login on W&B

In [ ]:
from google.colab import files
API_KEY = "9558532ae8e293df0f5981ae03aedf7d5ba0505a"
!wandb login --relogin $API_KEY

##Baseline

In [ ]:
%cd SICK_on_tweetsumm/src
!CUDA_VISIBLE_DEVICES="0" python3 train_summarization_base.py --finetune_weight_path="./new_weights_base" --best_finetune_weight_path="./new_weights_base_best" --dataset_name="tweetsumm" --model_name="facebook/bart-large"  --epoch=25 --train_batch_size=10

In [ ]:
!zip tweetsumm_base_weights.zip new_weights_base new_weights_base_best
files.download("tweetsumm_base_weights.zip")

###Inference

In [ ]:
!CUDA_VISIBLE_DEVICES="0" python3 inference.py --dataset_name "tweetsumm" --model_checkpoint="./new_weights_base_best" --test_output_file_name="./base_tmp_result.txt" --num_beams 20 --train_configuration="base"
files.download("base_tmp_result.txt")
%cd ../..

##Finetuning SICK

In [ ]:
%cd SICK_on_tweetsumm/src
!CUDA_VISIBLE_DEVICES="0" python3 train_summarization_context.py --finetune_weight_path="./new_weights_sick" --best_finetune_weight_path="./new_weights_sick_best" --dataset_name="tweetsumm" --model_name="facebook/bart-large" --epoch=25 --use_sentence_transformer True --train_batch_size=10

In [ ]:
!zip tweetsumm_sick_weights.zip new_weights_sick new_weights_sick_best
files.download("tweetsumm_sick_weights.zip")

###Inference

In [ ]:
!CUDA_VISIBLE_DEVICES="0" python3 inference.py --dataset_name "tweetsumm" --model_checkpoint="./new_weights_sick_best" --test_output_file_name="./sick_tmp_result.txt" --num_beams 20 --train_configuration="context" --use_sentence_transformer True
files.download("sick_tmp_result.txt")
%cd ../..

##Finetuning SICK++

In [ ]:
%cd SICK_on_tweetsumm/src
!CUDA_VISIBLE_DEVICES="0" python3 train_summarization_full.py --finetune_weight_path="./new_weights_sickplus" --best_finetune_weight_path="./new_weights_sickplus_best" --dataset_name="tweetsumm" --model_name="facebook/bart-large" --epoch=25 --use_sentence_transformer True --train_batch_size=5

 56% 1231/2200 [38:00<26:00,  1.61s/it]

In [ ]:
!zip tweetsumm_sickplus_weights.zip new_weights_sickplus new_weights_sickplus_best
files.download("tweetsumm_sickplus_weights.zip")

###Inference

In [ ]:
!CUDA_VISIBLE_DEVICES="0" python3 inference.py --dataset_name "tweetsumm" --model_checkpoint="./new_weights_sickplus_best" --test_output_file_name="./sickplus_tmp_result.txt" --num_beams 20 --train_configuration="full" --use_sentence_transformer True
files.download("sickplus_tmp_result.txt")
%cd ../..